# Spotify API

<img src = "https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_RGB_Green.png">

In [6]:
import requests
import pandas as pd
import base64

CLIENT_ID = '7b573e9481624f4a96e8b3ef917c8959'
CLIENT_SECRET = '631ff63e8d54494c914b320fc27c9e85'

class Artist:
    
    
    def __init__(self, name_artist):
        self._name_artist = name_artist.replace(' ', '+').lower()
        self._url_base = f'https://api.spotify.com/v1'
        self._artist_id = self._get_artist_id()
        self._albums = self._get_albums()
        
        # Discography of the artist
        self.discography = self._get_discography()
    
    
    def _get_token(self):
        token_encoded = base64.b64encode(bytes(f'{CLIENT_ID}:{CLIENT_SECRET}', 'utf-8'))
        token_url = 'https://accounts.spotify.com/api/token'
        params = {'grant_type' : 'client_credentials'}
        header_post = {'Authorization' : f'Basic {str(token_encoded, "utf-8")}'}
        request = requests.post(token_url, data=params, headers=header_post)
        if request.status_code == 200:
            token = request.json()['access_token']
            return token
        else:
            print(f'Error :{request.json()}')
    
    
    def _get_artist_id(self):
        params = {'q':self._name_artist, 'type':'artist', 'market':'US'}
        header_get = {'Authorization' : f'Bearer {self._get_token()}'}
        request = requests.get(f'{self._url_base}/search', headers=header_get, params=params)
        df_results = pd.DataFrame(request.json()['artists']['items'])
        df_results = df_results.sort_values(by='popularity', ascending=False)
        artist_id = df_results.iloc[0]['id']
        
        return artist_id
    
    
    def _get_albums(self):
        url_albums = f'{self._url_base}/artists/{self._artist_id}/albums'
        params = {'country' : 'US'}
        header_get = {'Authorization' : f'Bearer {self._get_token()}'}
        request_albums = requests.get(url_albums, headers=header_get, params=params)
        if request_albums.status_code == 200:
            artist_albums = request_albums.json()['items']
            albums = [[album['id'], album['name']] for album in artist_albums]
            return albums
        else:
            print(f'There was an error: {request_albums.status_code}')
    
    
    def _get_discography(self):
        discography = []
        
        for album in self._albums:
            url_album = f'{self._url_base}/albums/{album[0]}/tracks'
            header_get = {'Authorization' : f'Bearer {self._get_token()}'}
            params = {
                'country' : 'US',
                'limit' : 50,
                'offset' : 0,
            }
            album_request = requests.get(url_album, headers=header_get, params=params)
            if album_requestm.status_code == 200:
                album_list = album_request.json()['items']
                tracks = [track['name'] for track in album_list]
                album = {
                    'Album' : album[1],
                    'tracks' : tracks,
                }
                discography.append(album)
                return discography
            else:
                print(f'There was an error: {album_request.status_code}')

In [7]:
artist = Artist('Slipknot')

r = artist.discography
for album in r:
    print(album)
    print('\n')

{'Album': 'We Are Not Your Kind', 'tracks': ['Insert Coin', 'Unsainted', 'Birth Of The Cruel', 'Death Because of Death', 'Nero Forte', 'Critical Darling', "A Liar's Funeral", 'Red Flag', "What's Next", 'Spiders', 'Orphan', 'My Pain', 'Not Long for This World', 'Solway Firth']}


{'Album': 'Day Of The Gusano (Live)', 'tracks': ['Sarcastrophe - Live', 'The Heretic Anthem - Live', 'Psychosocial - Live', 'The Devil In I - Live', 'Me Inside - Live', 'Vermilion - Live', 'Wait And Bleed - Live', 'Prosthetics - Live', 'Before I Forget - Live', 'Eeyore - Live', 'Duality - Live', 'Custer - Live', 'Spit It Out - Live', 'Metabolic / 742617000027 - Live', '(Sic) - Live', 'People = Shit - Live', 'Surfacing - Live']}


{'Album': '.5: The Gray Chapter (Special Edition)', 'tracks': ['XIX', 'Sarcastrophe', 'AOV', 'The Devil in I', 'Killpop', 'Skeptic', 'Lech', 'Goodbye', 'Nomadic', 'The One That Kills the Least', 'Custer', 'Be Prepared for Hell', 'The Negative One', 'If Rain Is What You Want', 'Override